In [1]:
import os
import time
import torch
import torchtuples as tt
import numpy as np
import easydict
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold
import os
import time
import torch
import torchtuples as tt
import numpy as np
import easydict
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold

# Preprocessing

In [2]:
df = pd.read_csv('Dataset/processdata.csv', encoding='latin-1')
data = df

date_columns = ['Date.of.Last.Contact', 'Date.of.Diagnostic']
data[date_columns] = data[date_columns].apply(pd.to_datetime, errors='coerce')
has_na = data[date_columns].isna().any(axis=1)

# My comment: why not death time - contact time?
if has_na.any():
    data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
else:
    data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
    
data.loc[:, 'Survival_Time'] = data['Survival_Time'].replace({-1:0})

data['indicater'] = np.where(data['Date.of.Death'].isna(), 0, 1)
columns_to_drop = ['Date.of.Death', 'Date.of.Last.Contact', 'Date.of.Diagnostic']


data.drop(columns=columns_to_drop, inplace=True)

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler


columns_to_one_hot = ['RCBP.Name', 'Raca.Color', 'State.Civil', 'Code.Profession', 'Name.Occupation', 'Status.Address',
                      'City.Address', 'Description.of.Topography', 'Topography.Code', 'Morphology.Description',
                      'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Diagnostic.means', 'Extension',
                      'Type.of.Death']

# Replace other values that are not in top 9, into "other"
for column in columns_to_one_hot:
    top_9_values = data[column].value_counts().nlargest(9).index
    data[column] = data[column].where(data[column].isin(top_9_values), 'other')

data = pd.get_dummies(data, columns=columns_to_one_hot)

columns_to_binarize = ['Gender', 'Indicator.of.Rare.Case']

lb = LabelBinarizer()
for column in columns_to_binarize:
    data[column] = lb.fit_transform(data[column])

scaler = MinMaxScaler()
data['Age'] = scaler.fit_transform(data[['Age']])



X = data.drop(['Survival_Time', 'indicater'], axis=1)
time_all = data['Survival_Time'].values
event_all = data['indicater'].values
max_time = data['Survival_Time'].max()
print(max_time)

data.head()

7408


,Gender,Age,Indicator.of.Rare.Case,Survival_Time,indicater,RCBP.Name_RCBP BELO HORIZONTE,RCBP.Name_RCBP CAMPINAS-UNICAMP,RCBP.Name_RCBP DISTRITO FEDERAL,RCBP.Name_RCBP DRS BARRETOS,RCBP.Name_RCBP FORTALEZA,...,Diagnostic.means_PESQUISA,Diagnostic.means_SDO,Diagnostic.means_other,Extension_IN SITU,Extension_LOCALIZADO,Extension_MET?STASE,Extension_N?O SE APLICA,Type.of.Death_C?NCER,Type.of.Death_N?O C?NCER,Type.of.Death_other
0,1,0.283019,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
1,1,0.490566,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
2,1,0.698113,0,0,1,False,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
3,1,0.556604,0,0,1,False,True,False,False,False,...,False,True,False,False,False,True,False,True,False,False
4,0,0.122642,0,3959,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True


In [3]:
import numpy as np

# Take the maximum time and divide it into any number of intervals
Tmax = 7500
num_intervals = 7

intervals = [(i * (Tmax // num_intervals), (i + 1) * (Tmax // num_intervals)) for i in range(num_intervals)]

Y = np.zeros((len(time_all), num_intervals), dtype=np.int_)

# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_all):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y[i, j] = 1

Y = torch.Tensor(Y)

print(Y.shape)

torch.Size([14755, 7])


In [4]:
data[data['Survival_Time']>1000]

,Gender,Age,Indicator.of.Rare.Case,Survival_Time,indicater,RCBP.Name_RCBP BELO HORIZONTE,RCBP.Name_RCBP CAMPINAS-UNICAMP,RCBP.Name_RCBP DISTRITO FEDERAL,RCBP.Name_RCBP DRS BARRETOS,RCBP.Name_RCBP FORTALEZA,...,Diagnostic.means_PESQUISA,Diagnostic.means_SDO,Diagnostic.means_other,Extension_IN SITU,Extension_LOCALIZADO,Extension_MET?STASE,Extension_N?O SE APLICA,Type.of.Death_C?NCER,Type.of.Death_N?O C?NCER,Type.of.Death_other
4,0,0.122642,0,3959,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
14,0,0.811321,0,3595,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
15,0,0.415094,0,1727,1,False,True,False,False,False,...,False,False,False,True,False,False,False,True,False,False
33,0,0.594340,0,3624,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
34,0,0.698113,0,1808,0,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14747,1,0.433962,0,1313,1,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
14748,0,0.518868,0,1147,1,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
14749,1,0.707547,0,1134,1,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
14750,0,0.613208,0,1475,1,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


In [5]:
data['indicater'].value_counts()

indicater
1    10210
0     4545
Name: count, dtype: int64

In [6]:
data[data['indicater']==0]

,Gender,Age,Indicator.of.Rare.Case,Survival_Time,indicater,RCBP.Name_RCBP BELO HORIZONTE,RCBP.Name_RCBP CAMPINAS-UNICAMP,RCBP.Name_RCBP DISTRITO FEDERAL,RCBP.Name_RCBP DRS BARRETOS,RCBP.Name_RCBP FORTALEZA,...,Diagnostic.means_PESQUISA,Diagnostic.means_SDO,Diagnostic.means_other,Extension_IN SITU,Extension_LOCALIZADO,Extension_MET?STASE,Extension_N?O SE APLICA,Type.of.Death_C?NCER,Type.of.Death_N?O C?NCER,Type.of.Death_other
4,0,0.122642,0,3959,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
14,0,0.811321,0,3595,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
33,0,0.594340,0,3624,0,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
34,0,0.698113,0,1808,0,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
35,0,0.566038,0,2567,0,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14163,0,0.622642,0,3449,0,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
14164,0,0.622642,0,3450,0,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
14165,1,0.660377,0,4789,0,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
14166,1,0.679245,0,3303,0,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True


In [7]:
# Creat mask matrix
W = np.zeros((len(time_all), num_intervals), dtype=np.int_)

# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_all, event_all)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W[i, j] = 0
        else:
            W[i, j] = 1

W = torch.Tensor(W)

In [8]:
# change the batch size from 1024 to 3222 to improve the performance
# created by Xinyu modified by Jingyan
args = easydict.EasyDict({
    "batch_size": 2048,
    "cuda": True, # should set it to be true when using gpu, otherwise data would be on two devices
    "lr": 0.05,
    "seed": 1111,
    "reduce_rate": 0.95,
    "epochs": 20,
    "clip": 5.0,
    "log_interval":10,
})

In [9]:
for col in X.columns:
    if X[col].dtype == bool:
        X[col] = X[col].astype(int)
X_use = torch.tensor(X.values, dtype=torch.float32)
X_use.shape

torch.Size([14755, 138])

In [10]:
# Convert n*m data into m * n * 1 
Y_transform = [Y[:, i:i+1] for i in range(Y.size(1))]
print(Y_transform[0].shape)
W_transform = [W[:, i:i+1] for i in range(W.size(1))]
print(W_transform[0].shape)

torch.Size([14755, 1])
torch.Size([14755, 1])


In [11]:
# Jingyan adds the dataloader
from torch.utils.data import Dataset, DataLoader, random_split

class MultiTaskDataset(Dataset):
    def __init__(self, data, targets, masks, event_all):
        self.data = data
        self.targets = targets
        self.masks = masks
        self.event_all = event_all

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], [self.targets[i][idx] for i in range(len(self.targets))], [self.masks[i][idx] for i in range(len(self.masks))], self.event_all[idx]


#Xinyu
full_dataset = MultiTaskDataset(X_use, Y_transform, W_transform, event_all)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)

In [12]:
for x, y, w, e in train_loader:
    print(x.shape)
    print(len(y), y[0].shape)
    print(len(w), w[0].shape)
    print(e.shape)
    in_features = x.shape[1]
    out_features = len(y)
    break

torch.Size([2048, 138])
7 torch.Size([2048, 1])
7 torch.Size([2048, 1])
torch.Size([2048])


# Model

In [14]:
import torch.nn as nn
import torch.nn.functional as F


# Made by Xinyu and modified by Dr.Li and Jingyan
class MultiTaskModel(nn.Module):
    def __init__(self, in_features, out_features):
        super(MultiTaskModel, self).__init__()
        self.num_tasks = out_features
        self.input_features = in_features

        self.shared_layers = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1),

            nn.Linear(128, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1),

            nn.Linear(128, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1)
        )
        # A Classic Multi Task Learning Framework
        self.task_layers = nn.ModuleList([nn.Linear(128, 1) for _ in range(out_features)])#after your papaer I decided to use MTL to solve this prob

    # This forward propagation logic defines the chain propagation of our idea
    def forward(self, x):
        shared_output = self.shared_layers(x)
        task_outputs = []
        for i, task_layer in enumerate(self.task_layers):
            if i == 0:
                task_output = torch.sigmoid(task_layer(shared_output))
            else:
                task_output = torch.sigmoid(task_layer(shared_output)) * task_outputs[-1]
            task_outputs.append(task_output)

        return task_outputs# Here I difined the S(x)
    
    # Modified by Jingyan
    def custom_loss(self, task_outputs, targets, masks):
        loss = 0
        for i, task_output in enumerate(task_outputs):
            task_target = targets[i]
            task_mask = masks[i]
            task_loss = F.binary_cross_entropy(task_output, task_target.float(), reduction='none')
            task_loss = task_loss * task_mask.float() # [2048, 1]
            # print('task_loss', task_loss.shape)
            loss += task_loss.sum() / task_mask.sum()
        return loss








# Training

A few suggestions:

- To get better results, I suggest using cross-validation to find the best values for hyperparameters like `lambda_reg`.
- It is better to use a learning rate scheduler, early stopping, and model checkpointing to get the best model during the training process.


## Training with Gradients as attribution

In [16]:
from tqdm import tqdm, trange
from torch.autograd import grad
from torch.optim.lr_scheduler import ReduceLROnPlateau



def compute_gradients(features, task_output):
        inp_grad = grad(outputs=torch.log(task_output),
                        inputs=features,
                        grad_outputs=torch.ones_like(task_output),
                        create_graph=True)[0]
        return inp_grad

device = torch.device("cuda" if args.cuda else "cpu")
model = MultiTaskModel(in_features, out_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lambda_reg = 0.1


for epoch in trange(args.epochs):
    running_loss = 0.0
    for batch_idx, (X_train, targets, masks, event_train) in enumerate(train_loader):
        
        X_train = X_train.to(device)
        X_train.requires_grad = True
        targets = [target.to(device) for target in targets]
        masks = [mask.to(device) for mask in masks]
        optimizer.zero_grad()
        task_outputs = model(X_train)
        # Base loss computation
        L_base = model.custom_loss(task_outputs, targets, masks)
        
        Omega_smooth = 0
        prev_attributions = None
        for t in range(out_features):
            current_attributions = compute_gradients(X_train, task_outputs[t])
            # Compute smoothing regularization
            if prev_attributions is not None:
                diff = current_attributions - prev_attributions
                Omega_smooth += torch.abs(diff)              
            prev_attributions = current_attributions
            
        # The formula in project description is summation, but I got better result by taking average.
        Omega_smooth = torch.sum(Omega_smooth)  # Omega_smooth.mean() #torch.sum(Omega_smooth) 
        loss = L_base + (lambda_reg * Omega_smooth)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f'End of Epoch {epoch}, Average Loss: {avg_loss:.4f}')
        
    
    
    
    
    
    
    
    
    

  5%|▌         | 1/20 [00:00<00:11,  1.62it/s]

End of Epoch 0, Average Loss: 122.3635


 10%|█         | 2/20 [00:01<00:11,  1.60it/s]

End of Epoch 1, Average Loss: 79.6817


 15%|█▌        | 3/20 [00:01<00:09,  1.77it/s]

...

 20%|██        | 4/20 [00:02<00:08,  1.81it/s]

End of Epoch 3, Average Loss: 40.8860


 25%|██▌       | 5/20 [00:02<00:08,  1.70it/s]

End of Epoch 4, Average Loss: 30.7131


KeyboardInterrupt: 

## Training with Expected Gradients

I used the edited package of Expected Gradients by Jingyan, but with a very minor change in the `shap_value` function.

In [15]:
from expected_gradient_multi_task import AttributionPriorExplainer
from tqdm import tqdm, trange
from torch.autograd import grad
from torch.optim.lr_scheduler import ReduceLROnPlateau



APExp = AttributionPriorExplainer(train_dataset,args.batch_size)



device = torch.device("cuda" if args.cuda else "cpu")
model = MultiTaskModel(in_features, out_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lambda_reg = 0.5


for epoch in trange(args.epochs):
    running_loss = 0.0
    for batch_idx, (X_train, targets, masks, event_train) in enumerate(train_loader):
        
        X_train = X_train.to(device)
        X_train.requires_grad = True
        targets = [target.to(device) for target in targets]
        masks = [mask.to(device) for mask in masks]
        optimizer.zero_grad()
        task_outputs = model(X_train)
        
        ## get Expected Gradients attribution
        eg = APExp.shap_values(model,X_train)
        
        # Base loss computation
        L_base = model.custom_loss(task_outputs, targets, masks)
        
        Omega_smooth = 0
        prev_attributions = None
        for t in range(out_features):
            current_attributions = eg[t]
            # Compute smoothing regularization
            if prev_attributions is not None:
                diff = current_attributions - prev_attributions
                Omega_smooth += torch.abs(diff)              
            prev_attributions = current_attributions
            
        
        Omega_smooth = torch.sum(Omega_smooth)  # Omega_smooth.mean() #torch.sum(Omega_smooth) 
        loss = L_base + (lambda_reg * Omega_smooth)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f'End of Epoch {epoch}, Average Loss: {avg_loss:.4f}')
        
    

  5%|▌         | 1/20 [00:02<00:48,  2.57s/it]

End of Epoch 0, Average Loss: 16.5027


 10%|█         | 2/20 [00:03<00:30,  1.67s/it]

End of Epoch 1, Average Loss: 12.0195


 15%|█▌        | 3/20 [00:04<00:24,  1.42s/it]

End of Epoch 2, Average Loss: 9.5268


 15%|█▌        | 3/20 [00:05<00:31,  1.84s/it]


KeyboardInterrupt: 

# Evaluation

In [29]:
# Created by Jingyan
model.eval()
test_predictions = []

# Disable gradient calculations
with torch.no_grad():
    for X_test, _, _, _ in test_loader:
        # Forward pass
        X_test = X_test.to(device)
        task_outputs_ = model(X_test)

        # Store the predictions
        test_predictions.append(task_outputs_)
test_predictions = [torch.cat([preds[i] for preds in test_predictions]) for i in range(len(test_predictions[0]))]



In [30]:
# Created by Jingyan
model.eval()
train_predictions = []

# Disable gradient calculations
with torch.no_grad():
    for X_train, _, _, _ in train_loader:
        # Forward pass
        X_train = X_train.to(device)
        task_outputs_ = model(X_train)

        # Store the predictions
        train_predictions.append(task_outputs_)

# Process the predictions as needed
# For example, converting them to a list or concatenating
# Here we concatenate the predictions for each task
train_predictions = [torch.cat([preds[i] for preds in train_predictions]) for i in range(len(train_predictions[0]))]



In [31]:
# made by Xinyu
def binarize_and_sum_columns(output_list):
    def binarize_list(input_list):
        tensor = torch.Tensor(input_list)
        # print(input_list.max() == input_list.min())
        binary_tensor = (tensor >= 0.5).float()
        return binary_tensor

    result = binarize_list(output_list[0])
    for i in range(1, len(output_list)):
        binary_column = binarize_list(output_list[i])
        # print(binary_column.max() == binary_column.min())
        result += binary_column

    return result



In [33]:
# made by Xinyu
Y_hat_train = binarize_and_sum_columns(train_predictions)
Y_hat_train = Y_hat_train.squeeze()
print(Y_hat_train.shape)

Y_hat_test = binarize_and_sum_columns(test_predictions)
Y_hat_test = Y_hat_test.squeeze()
# print(Y_hat_test.min(), Y_hat_test.max())

torch.Size([10240])


In [29]:
# created by Jingyan
# Get the true labels and status
train_trues = []
train_statuses = []
for _, train_targets, train_masks, train_status in train_loader:
    true_label = [train_targets[i]*train_masks[i] for i in range(len(train_targets))]
    train_trues.append(true_label)
    train_statuses.append(train_status)
train_trues = [torch.cat([preds[i] for preds in train_trues]) for i in range(len(train_trues[0]))]
train_statuses = torch.cat([status for status in train_statuses])

print(len(train_trues))
print(train_trues[0].shape)
print(train_statuses.shape)

test_trues = []
test_statuses = []
for _, test_targets, test_masks, test_status in test_loader:
    true_label = [test_targets[i]*test_masks[i] for i in range(len(test_targets))]
    test_trues.append(true_label)
    test_statuses.append(test_status)

test_trues = [torch.cat([preds[i] for preds in test_trues]) for i in range(len(test_trues[0]))]
test_statuses = torch.cat([status for status in test_statuses])

print(len(test_trues))
print(test_trues[0].shape)
print(test_statuses.shape)

7
torch.Size([10240, 1])
torch.Size([10240])
7
torch.Size([2048, 1])
torch.Size([2048])


In [23]:
# made by Xinyu
Y_true_train = binarize_and_sum_columns(train_trues)
Y_true_train = Y_true_train.squeeze()
print(Y_true_train.shape)

Y_true_test = binarize_and_sum_columns(test_trues)
Y_true_test = Y_true_test.squeeze()
print(Y_true_test.shape)

torch.Size([10240])
torch.Size([2048])


In [193]:
class Cindex(torch.nn.Module):
    def __init__(self):
        super(Cindex, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.Tensor(y)
        if not torch.is_tensor(y_hat):
            y_hat = torch.Tensor(y_hat)
        if not torch.is_tensor(status):
          status = torch.Tensor(status)

        N = y.size(0)
        total_pairs = 0
        c = 0

        for i in trange(N):
            for j in range(i + 1, N):
                a = y[i]
                b = y[j]
                a_hat = y_hat[i]
                b_hat = y_hat[j]
                astatus = status[i]
                bstatus = status[j]
                if (a >= b and a_hat >= b_hat and bstatus == 1) or (a <= b and a_hat <= b_hat and astatus == 1):
                    c += 1
                if (a <= b and astatus==1) or (b <= a and bstatus == 1):
                    total_pairs += 1

        outcome = c / total_pairs
        return outcome

cindex_calculator = Cindex()
# c11_train = cindex_calculator(Y_true_train, Y_hat_train, train_statuses)
# print(c11_train)
c11_test = cindex_calculator(Y_true_test, Y_hat_test, test_statuses)
print(c11_test)

100%|██████████| 2048/2048 [03:20<00:00, 10.23it/s] 

0.9783050763370184
